**Credit Card Fraud Detection - Classification**
### Anonymized credit card transactions labeled as fraudulent or genuine


---


The dataset contains transactions made by credit cards in September 2013 by European cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.


---


**Context**

It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.


---


**Content**

The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

In [ ]:
# Importing Required Lib
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# reading the data set
data_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/creditcard.csv')

In [ ]:
#showing top 5 records
data_df.head()

In [ ]:
# Checking shape of data set
data_df.shape

In [ ]:
# Checking Data Types
data_df.dtypes

In [ ]:
# Checking Null values
data_df.isnull().sum()

In [ ]:
# NO null values

In [ ]:
# Duplicate Entry
data_df.duplicated().sum()

In [ ]:
# We have 1081 Duplicate Entry out of 284807 entry. Let remove it first before moving forward. 

data_df.drop_duplicates(inplace = True)

In [ ]:
data_df.duplicated().sum()

In [ ]:
# Let Describe our data
data_df.describe()

In [ ]:
# The classes are heavily skewed we need to solve this issue later.
print('No Frauds', round(data_df['Class'].value_counts()[0]/len(data_df) * 100,2), '% of the dataset')
print('Frauds', round(data_df['Class'].value_counts()[1]/len(data_df) * 100,2), '% of the dataset')

**Note:**  Most of the transactions are non-fraud. If we use this dataframe as the base for our predictive models and analysis we might get a lot of errors and our algorithms will probably overfit since it will "assume" that most transactions are not fraud. But we don't want our model to assume, we want our model to detect patterns that give signs of fraud!

In [ ]:
sb.countplot(data = data_df, x = 'Class')
plt.title('Class Distributions \n (0 -- No Fraud & 1 --Fraud)')
plt.show()

In [ ]:
plt.figure(figsize = (30,15))
sb.heatmap(data_df.corr(), annot = True)

In [ ]:
X = data_df[['Amount', 'Time']]
Y = data_df['Class']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test, Y_train,Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 1)

In [ ]:
Y_train.value_counts()

In [ ]:
# Converts minority class to majority class

from imblearn.over_sampling import RandomOverSampler

In [ ]:
ros = RandomOverSampler(random_state= 1)

In [ ]:
X_train1, Y_train1 = ros.fit_resample(X_train, Y_train)

In [ ]:
X_test1, Y_test1 = ros.fit_resample(X_test,Y_test)

In [ ]:
Y_train1.value_counts()

In [ ]:
Y_test1.value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
ss = StandardScaler()
X_train1 = ss.fit_transform(X_train1)
X_test1 = ss.transform(X_test1)

In [ ]:
# Creat model
model = tf.keras.Sequential([
    # First Layer
    tf.keras.layers.Dense(units = 64, activation = 'relu', input_shape = (X.shape[1],)),
    # Second Layer
    tf.keras.layers.Dense(units = 32, activation = 'relu'),
    # Third Layer
    tf.keras.layers.Dense(units = 16, activation = 'relu'),
    # Fourth Layer
    tf.keras.layers.Dense(units = 8, activation = 'relu'),
    # output
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid'),
                              ])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer ='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# create a procedure for EARLY STOPING
# calling inbuilt class:   EarlyStopping

from tensorflow.keras.callbacks import EarlyStopping

#create object  
cb = EarlyStopping(monitor = 'val_loss',   # Mention parameter to monitor .. it may me loss or score
                   min_delta = 0.0001,     # diff btw new and previous loss: bydefault we take 0.0001
                   patience = 20,          
                   verbose =1,
                   mode = 'auto',
                   baseline = None,
                   restore_best_weights =False)

In [ ]:
# train the model: use inbuilt method fit() of Sequential class

train_model = model.fit(X_train1, Y_train1,epochs =5000,validation_data=(X_test1,Y_test1), callbacks = cb)

In [ ]:
# Here we see that out model is Underfit ....No we will increse the no of Hidden layer
print("Traning Loss and Accuracy: ", model.evaluate(X_train1, Y_train1))
print("Testing Loss and Accuracy: ", model.evaluate(X_test1, Y_test1))

In [ ]:
# Visulisation
plt.plot(train_model.history['loss'],label = 'Traning Loss')
plt.plot(train_model.history['val_loss'],label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('binary_crossentropy')
plt.legend()
plt.show()

In [ ]:
# Visulisation
plt.plot(train_model.history['accuracy'],label = 'Traning Loss')
plt.plot(train_model.history['val_accuracy'],label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

**Here we see that our model is Overfit.Now we use dropout to reduce this overfiting.**

In [ ]:
# Create model with Dropout

from tensorflow.keras.layers import Dropout
model_1 = tf.keras.Sequential([
    # First Layer
    tf.keras.layers.Dense(units = 64, activation = 'relu', input_shape = (X.shape[1],)),Dropout(0.25),
    # Second Layer
    tf.keras.layers.Dense(units = 32, activation = 'relu'),Dropout(0.25),
    # Third Layer
    tf.keras.layers.Dense(units = 16, activation = 'relu'),Dropout(0.25),
    # Fourth Layer
    tf.keras.layers.Dense(units = 8, activation = 'relu'),Dropout(0.25),
    # output
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid'),
                              ])

In [ ]:
model_1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# train the model: use inbuilt method fit() of Sequential class

train_model = model_1.fit(X_train1, Y_train1,epochs =5000,validation_data=(X_test1,Y_test1), callbacks = cb)

In [ ]:
# Here we see that out model is Underfit ....No we will increse the no of Hidden layer
print("Traning Loss and Accuracy: ", model_1.evaluate(X_train1, Y_train1))
print("Testing Loss and Accuracy: ", model_1.evaluate(X_test1, Y_test1))

In [ ]:
# Visulisation
plt.plot(train_model.history['loss'],label = 'Traning Loss')
plt.plot(train_model.history['val_loss'],label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('binary_crossentropy')
plt.legend()
plt.show()

In [ ]:
# Visulisation
plt.plot(train_model.history['accuracy'],label = 'Traning Loss')
plt.plot(train_model.history['val_accuracy'],label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

**Still it look like overfit model. Now we use regularizer with Dropout.**

In [ ]:
# Creat model

from tensorflow.keras import regularizers
model_2 = tf.keras.Sequential([
    # First Layer
    tf.keras.layers.Dense(units = 64, activation = 'relu',
                          kernel_regularizer = regularizers.l2(0.01),input_shape = (X.shape[1],)),Dropout(0.3),
    # Second Layer
    tf.keras.layers.Dense(units = 32, activation = 'relu',kernel_regularizer = regularizers.l2(0.01)),Dropout(0.3),
    # Third Layer
    tf.keras.layers.Dense(units = 16, activation = 'relu',kernel_regularizer = regularizers.l2(0.01)),Dropout(0.3),
    # fourth Layer
    tf.keras.layers.Dense(units = 8, activation = 'relu',kernel_regularizer = regularizers.l2(0.01)),Dropout(0.3),
    # output
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid',kernel_regularizer = regularizers.l2(0.01)),
                              ])

In [ ]:
model_2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# train the model: use inbuilt method fit() of Sequential class

train_model = model_2.fit(X_train1, Y_train1,epochs =5000,validation_data=(X_test1,Y_test1), callbacks = cb)

In [ ]:
# Here we see that out model is Underfit ....No we will increse the no of Hidden layer
print("Traning Loss and Accuracy: ", model_2.evaluate(X_train1, Y_train1))
print("Testing Loss and Accuracy: ", model_2.evaluate(X_test1, Y_test1))

In [ ]:
# Visulisation
plt.plot(train_model.history['accuracy'],label = 'Traning Loss')
plt.plot(train_model.history['val_accuracy'],label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Visulisation
plt.plot(train_model.history['loss'],label = 'Traning Loss')
plt.plot(train_model.history['val_loss'],label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('binary_crossentropy')
plt.legend()
plt.show()